In [42]:
import gensim
import numpy as np

# Load pre-trained word vectors

In [323]:
import gc
gc.collect()

274

In [ ]:
# Google news pretrained vectors
wv_model_path = "/home/v2john/Documents/GoogleNews-vectors-negative300.bin.gz"
wv_model = gensim.models.KeyedVectors.load_word2vec_format(wv_model_path, binary=True, unicode_errors='ignore')

# Twitter pretrained vectors
wv_model_path_1 = "/home/v2john/Documents/word2vec_twitter_model/word2vec_twitter_model.bin"
wv_model_1 = gensim.models.KeyedVectors.load_word2vec_format(wv_model_path_1, binary=True, unicode_errors='ignore')

In [297]:
w2v_dimensions = len(wv_model['word'])
print(w2v_dimensions)

300


In [298]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

lmtzr = WordNetLemmatizer()

def remove_stopwords(string):
    split_string = \
        [word for word in string.split()
         if word not in stopwords.words('english')]
    
    return " ".join(split_string)

In [299]:
import re

def clean_str(string):
    string = re.sub(r"@[A-Za-z0-9_s(),!?\'\`]+", "", string) # removing any twitter handle mentions
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)

    return remove_stopwords(string.strip().lower())

# Load affect intensity lexica

In [300]:
affect_intensity_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/" + \
    "lexicons/NRC-AffectIntensity-Lexicon.txt"

def get_word_affect_intensity_dict(emotion):
    word_intensities = dict()

    with open(affect_intensity_file_path) as affect_intensity_file:
        for line in affect_intensity_file:
            word_int_array = line.replace("\n", "").split("\t")

            if (word_int_array[2] == emotion):
                word_intensities[word_int_array[0]] = float(word_int_array[1])

    return word_intensities

In [301]:
sentiment_emotion_lex_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/" + \
    "lexicons/NRC-Sentiment-Emotion-Lexicons/Lexicons/NRC-Emotion-Lexicon-v0.92/" + \
    "NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"

def get_affect_presence_list(emotion):
    word_list = list()
    
    with open(sentiment_emotion_lex_file_path) as sentiment_emotion_lex_file:
        for line in sentiment_emotion_lex_file:
            word_array = line.replace("\n", "").split("\t")

            if (word_array[1] == emotion and word_array[2] == '1'):
                word_list.append(word_array[0])
                
    return word_list

In [302]:
hashtag_emotion_lex_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/" + \
    "lexicons/NRC-Sentiment-Emotion-Lexicons/Lexicons/NRC-Hashtag-Emotion-Lexicon-v0.2/" + \
    "NRC-Hashtag-Emotion-Lexicon-v0.2.txt"
    
def get_hashtag_emotion_intensity(emotion):
    hastag_intensities = dict()
    
    with open(hashtag_emotion_lex_file_path) as hashtag_emotion_lex_file:
        for line in hashtag_emotion_lex_file:
            word_array = line.replace("\n", "").split("\t")

            if (word_array[0] == emotion):
                hastag_intensities[word_array[1].replace("#", "")] = float(word_array[2])
                
    return hastag_intensities

In [303]:
print(clean_str("A night where depression is winning... #depression #fml #help"))

night depression winning depression fml help


In [304]:
class Tweet(object):

    def __init__(self, id, text, emotion, intensity):
        self.id = id
        self.text = text
        self.emotion = emotion
        self.intensity = intensity

    def __repr__(self):
        return \
            "id: " + self.id + \
            ", text: " + self.text + \
            ", emotion: " + self.emotion + \
            ", intensity: " + self.intensity

In [305]:
def read_training_data(training_data_file_path):

    train_list = list()
    with open(training_data_file_path) as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            train_list.append(Tweet(array[0], clean_str(array[1]), array[2], float(array[3])))
    return train_list
            
def read_test_data(training_data_file_path):

    test_list = list()
    with open(training_data_file_path) as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            test_list.append(Tweet(array[0], clean_str(array[1]), array[2], None))
    return test_list

In [306]:
emotion = "joy"

In [307]:
training_data_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/dataset/" + \
    emotion + "-ratings-0to1.train.txt"
test_data_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/dataset/" + \
    emotion + "-ratings-0to1.dev.target.txt"
predictions_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/predictions/" + \
    emotion + "-pred.txt"
gold_set_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/dataset/gold-set/" + \
    emotion + "-ratings-0to1.dev.gold.txt"

In [308]:
training_tweets = read_training_data(training_data_file_path)

score_train = list()
tweet_train = list()
for tweet in training_tweets:
    tweet_train.append(tweet.text)
    score_train.append(float(tweet.intensity))
print(len(score_train))

823


In [309]:
from sklearn.preprocessing import PolynomialFeatures

poly_emo_int = PolynomialFeatures(10)
word_intensities = get_word_affect_intensity_dict(emotion)

def get_emo_int_vector(tweet):
    score = 0.0
    for word in word_intensities.keys():
        if word in tweet:
            score += tweet.count(word) * float(word_intensities[word])
    
    return poly_emo_int.fit_transform(np.array([score]).reshape(1, -1))[0].tolist()

In [310]:
def get_word2vec_embedding(tweet):
    vector_list = list()
    for word in tweet.split():
        try:
            vector_list.append(wv_model[word])
        except Exception as e:
            pass

    if len(vector_list) == 0:
        vec_rep = np.zeros(w2v_dimensions).tolist()
    else:
        vec_rep = sum(vector_list) / float(len(vector_list))

    return vec_rep

In [311]:
word_list = get_affect_presence_list(emotion)

def get_sentiment_emotion_feature(tweet):
    for word in word_list:
        if word in tweet.split():
            return [1.0]
    
    return [0.0]

In [312]:
poly_emo_int = PolynomialFeatures(10)
hashtag_emotion_intensities = get_hashtag_emotion_intensity(emotion)

def get_hashtag_emotion_vector(tweet):
    score = 0.0
    for word in hashtag_emotion_intensities.keys():
        if word in tweet:
            score += tweet.count(word) * float(hashtag_emotion_intensities[word])
    
    return poly_emo_int.fit_transform(np.array([score]).reshape(1, -1))[0].tolist()

In [313]:
def vectorize_tweets(tweet_list):
    vectors = list()

    for tweet in tweet_list:
        x_vector = list()
        x_vector.extend(get_emo_int_vector(tweet))
        x_vector.extend(get_word2vec_embedding(tweet))
        x_vector.extend(get_sentiment_emotion_feature(tweet))
        x_vector.extend(get_hashtag_emotion_vector(tweet))
        vectors.append(x_vector)
    
    return vectors

In [314]:
x_train = vectorize_tweets(tweet_train)
print(len(x_train))
dimension = len(x_train[0])

with open("/tmp/dump.txt", 'w') as dump_file:
    for i in range(len(x_train)):
        if dimension != len(x_train[i]):
            print(len(x_train[i]), i)
        dump_file.write(str(x_train[i]))
        dump_file.write("\n")

823


In [315]:
test_tweets = read_test_data(test_data_file_path)
tweet_test = list()
for tweet in test_tweets:
    tweet_test.append(tweet.text)

x_test = vectorize_tweets(tweet_test)
print(len(x_test))

79


In [316]:
from sklearn import ensemble, svm

ml_model = ensemble.GradientBoostingRegressor(n_estimators=100)
ml_model.fit(x_train, score_train)

y_test = ml_model.predict(X=x_test)

In [317]:
y_gold = read_training_data(gold_set_path)

data_dict = dict()
diff = 0
for i in range(len(y_gold)):
    if y_gold[i].intensity >= 0.5:
        diff += y_gold[i].intensity - y_test[i]
#         print([tweet_test[i], str(y_test[i]), str(y_gold[i].intensity)])
print(diff/len(y_gold))

0.0654958123301


In [318]:
with open(predictions_file_path, 'w') as predictions_file:
    for i in range(len(y_test)):
        predictions_file.write(
            str(test_tweets[i].id) + "\t" + test_tweets[i].text + "\t" +
            test_tweets[i].emotion +"\t" + str(y_test[i]) + "\n"
        )

In [319]:
for i in range(10):
    print(tweet_test[i], y_test[i])

lol thought maybe , could n't decide levity 0.373060087771
nawaz sharif getting funnier day day laughter challenge kashmir baloch 0.552776455354
nawaz sharif getting funnier day day challenge kashmir baloch 0.497147744763
'll people watch enjoy rare show optimism 0.486934470901
love family much lucky grateful smartassfamily love 0.742502326897
love family much lucky grateful smartassfamily hilarious love 0.746048511005
assure laughter , increasing anger costs , arrogance westminster 0.390278405354
trump supporters hillary haters wanna chirp weak minded , pandering liberals tweet 0.199214901771
google caffeine sprightly lengthening corridor seo wgj 0.414636446228
tweet dedicated back pain , understand youthful spry full life vivacious 0.574428990027


# Evaluation based on Pearson and Spearman co-efficients

In [320]:
import numpy
import sys
import scipy.stats

def evaluate(pred,gold):
    
    f=open(pred, "rb")
    pred_lines=f.readlines()
    f.close()
    
    f=open(gold, "rb")
    gold_lines=f.readlines()
    f.close()
    

    if(len(pred_lines)==len(gold_lines)):       
        # align tweets ids with gold scores and predictions
        data_dic={}
        
        for line in gold_lines:
            line = line.decode()
            parts=line.split('\t')
            if len(parts)==4:   
                data_dic[int(parts[0])]=[float(line.split('\t')[3])]
            else:
                raise ValueError('Format problem.')
        
        
        for line in pred_lines:
            line = line.decode()
            parts=line.split('\t')
            if len(parts)==4:  
                if int(parts[0]) in data_dic:
                    try:
                        data_dic[int(parts[0])].append(float(line.split('\t')[3]))
                    except ValueError:
                        # Invalid predictions are replaced by a default value
                        data_dic[int(parts[0])].append(0.5)
                else:
                    raise ValueError('Invalid tweet id.')
            else:
                raise ValueError('Format problem.')
            
            
        
        # lists storing gold and prediction scores
        gold_scores=[]  
        pred_scores=[]
         
        
        # lists storing gold and prediction scores where gold score >= 0.5
        gold_scores_range_05_1=[]
        pred_scores_range_05_1=[]
         
            
        for id in data_dic:
            if(len(data_dic[id])==2):
                gold_scores.append(data_dic[id][0])
                pred_scores.append(data_dic[id][1])
                if(data_dic[id][0]>=0.5):
                    gold_scores_range_05_1.append(data_dic[id][0])
                    pred_scores_range_05_1.append(data_dic[id][1])
            else:
                raise ValueError('Repeated id in test data.')
                
      
        # return zero correlation if predictions are constant
        if numpy.std(pred_scores)==0 or numpy.std(gold_scores)==0:
            return (0,0,0,0)
        

        pears_corr=scipy.stats.pearsonr(pred_scores,gold_scores)[0]                                    
        spear_corr=scipy.stats.spearmanr(pred_scores,gold_scores)[0]   


        pears_corr_range_05_1=scipy.stats.pearsonr(pred_scores_range_05_1,gold_scores_range_05_1)[0]                                    
        spear_corr_range_05_1=scipy.stats.spearmanr(pred_scores_range_05_1,gold_scores_range_05_1)[0]           
        
      
        return (pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1)
                                           
                          
        
    else:
        raise ValueError('Predictions and gold data have different number of lines.')

In [321]:
print(emotion)
print("pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1")
print(evaluate(predictions_file_path, gold_set_path))

joy
pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1
(0.7164063316206607, 0.70389015766870533, 0.72058822972368297, 0.71857784219445264)


# Overall Score estimation

In [322]:
pears_corr_sum = 0
spear_corr_sum = 0
pears_corr_range_05_1_sum = 0
spear_corr_range_05_1_sum = 0

for emotion in ['anger', 'fear', 'sadness', 'joy']:
    print("\n" + emotion)
    predictions_file_path = \
        "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/predictions/" + \
        emotion + "-pred.txt"
    gold_set_path = \
        "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/dataset/gold-set/" + \
        emotion + "-ratings-0to1.dev.gold.txt"
    print("pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1")
    pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1 = \
        evaluate(predictions_file_path, gold_set_path)
    print(pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1)
    pears_corr_sum += pears_corr
    spear_corr_sum += spear_corr
    pears_corr_range_05_1_sum += pears_corr_range_05_1
    spear_corr_range_05_1_sum += spear_corr_range_05_1
    
print("\n===============================\n")
print("Average Scores")
print("pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1")
print(pears_corr_sum/4,spear_corr_sum/4,pears_corr_range_05_1_sum/4,spear_corr_range_05_1_sum/4)


anger
pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1
0.594034333503 0.488368217018 0.529485558742 0.435629743698

fear
pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1
0.640446207263 0.612774988643 0.490244553909 0.470890460112

sadness
pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1
0.516944362888 0.518882638684 0.356863335856 0.27446828507

joy
pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1
0.716406331621 0.703890157669 0.720588229724 0.718577842194


Average Scores
pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1
0.616957808819 0.580979000503 0.524295419558 0.474891582769


# Simple Neural Network Implementation in Keras

In [29]:
import theano

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
# define base model
_, dim_size = (np.array(x_train).shape)
print(dim_size)
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(500, activation='relu', input_dim=dim_size))
    model.add(Dropout(0.25))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(33, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=1000, batch_size=5, verbose=0)
estimator.fit(x_train, score_train)

In [ ]:
y_test = estimator.predict(x_test)
print(len(y_test))

In [ ]:
with open(predictions_file_path, 'w') as predictions_file:
    for i in range(len(y_test)):
        predictions_file.write(
            str(test_tweets[i].id) + "\t" + test_tweets[i].text + "\t" +
            test_tweets[i].emotion +"\t" + str(y_test[i]) + "\n"
        )